In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('formula1_daten.csv', sep = ';', decimal = '.')
del df['Unnamed: 0']
df.shape

(157755, 19)

Aufteilen des Datensatzes nach einzelnen Rennen und verkürzen der Datensätze nach prozentualem Rennfortschritt, zusätzlich wird eine Spalte hinzugefügt, die pro Runde die kumulierten Millisekunden enthält

In [17]:
#dictionary enthält einzelne datensätze zu jedem rennen
split_by_race = {}
#grenzwert ab dem vorhersage über rennausgang gemacht werden soll (orientiert sich an race_completion)
border = 0.4
#dictionary, welches renndatensätze nur bis zu einem gewissen zeitpunkt enthält (zp wird nach border gewählt)
sliced_races = {}

for rid in df['raceId'].unique():
    race = df.where(df.raceId == rid).dropna(how = 'all')
    race.reset_index(inplace = True, drop = True)
    split_by_race[rid] = race
    
    #finden der lap_number wo race_completion die angegebene border überschreitet
    last_lap_num = race.where(race.race_completion == min(race.where(race.race_completion >= border).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]
    
    #es werden nur daten aus race genommen, die bis zu dieser lap_number gehen
    race_shortened = race.where(race.lap_number < last_lap_num).dropna(how = 'all')
    race_shortened.reset_index(inplace = True, drop = True)
    
    #hinzufügen einer spalte die die insgesamt gefahrenen millisekunden enthält
    bis zu dieser Runde pro Fahrer enthält
    race_shortened['sum_milliseconds_pro_lap'] = 0
    
    for did in race_shortened.driverId.unique():
        for lapnum in race_shortened.lap_number.unique():
            sum_ms = np.sum(race_shortened.where(np.logical_and(race_shortened.driverId == did,race_shortened.lap_number<=lapnum)).dropna(how = 'all')['lap_in_milliseconds'])
            
            #setzen der bisher gefahrenen Zeit (kumuliert) pro Fahrer und Runde
            race_shortened.loc[race_shortened.where(np.logical_and(race_shortened.driverId == did,race_shortened.lap_number==lapnum)).dropna(how = 'all').index,'sum_milliseconds_pro_lap'] = sum_ms
    
    sliced_races[rid] = race_shortened
    
    
    

Anlegen von Ordnern, in denen DataFrames abgespeichert werden, um Zelle oben nicht immer ausführen zu müssen

In [ ]:
if not os.path.exists('sliced_data'):
    os.makedirs('sliced_data')
    
if not os.path.exists('split_data'):
    os.makedirs('split_data')

for key, value in sliced_races.items():
    name = 'sliced_data/sliced_'+str(int(key))+'.csv'
    value.to_csv(name,sep = ';', decimal = '.')
for key, value in split_by_race.items():
    name = 'split_data/split_'+str(int(key))+'.csv'
    value.to_csv(name,sep = ';', decimal = '.')

Laden der oben erzeugten DataFrames aus den erstellten Ordnern

In [ ]:
if os.path.exists('sliced_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/sliced_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    sliced_races = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('sliced_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('sliced_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        sliced_races[f] = df
    print('Einlesen der sliced Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')
    
if os.path.exists('split_data'):
    csv_filenames = []
    #auslesen aller csv file dateinamen aus formula 1 datensatz und abspeichern in liste
    for filename in os.listdir(os.getcwd()+'/split_data'):
        typ = filename.split('.')[-1]
        name = filename.split('.')[0]
        if typ == 'csv':
            csv_filenames.append(filename)
    split_by_race = {}
    #einlesen und abspeichern als dataframe aller dateien
    for file in csv_filenames:
        try:
            df = pd.read_csv('split_data/'+file, engine = 'python', sep = ';', decimal = '.')
            del df['Unnamed: 0']
        except Exception as e:
            df = pd.read_csv('split_data/'+file, engine = 'c', sep = ';', decimal = '.')
            del df['Unnamed: 0']
            print(e)
        #print(df.head())
        f = int(file.split('_')[-1].split('.')[0])
        split_by_race[f] = df
    print('Einlesen der split Dateien erfolgreich')
else:
    print('Dateien können nicht eingelesen werden, da kein entsprechendes Verzeichnis existiert!')

alle Daten werden in sliced_races, split_by_race gespeichert

In [20]:
sliced_races[841]['race_completion']

0      0.017241
1      0.034483
2      0.051724
3      0.068966
4      0.086207
         ...   
478    0.327586
479    0.344828
480    0.362069
481    0.379310
482    0.396552
Name: race_completion, Length: 483, dtype: float64

#### Anhang:

In [11]:
temp = split_by_race[841]

In [ ]:
#setzen der prozentualen Grenze, ab der weiterer Verlauf des Rennens vorhergesagt werden soll
border = 0.4

for race in split_by_race.values():
    
    #finden der lap_number wo race_completion die angegebene border überschreitet
    last_lap_num = race.where(race.race_completion == min(race.where(race.race_completion >= border).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]
    
    #es werden nur daten aus race genommen, die bis zu dieser lap_number gehen
    race_shortened = race.where(race.lap_number < last_lap_num).dropna(how = 'all')
    race_shortened.reset_index(inplace = True, drop = True)
    

In [16]:
temp.where(temp.race_completion == min(temp.where(temp.race_completion >= 0.4).dropna(how = 'all')['race_completion'])).dropna(how = 'all')['lap_number'].unique()[0]

24.0